Hello. Beautiful People~!
  
I'm not good enough, but I'm going to study the ensemble step by step.    
  
**We are only precious beings in the universe.**  
If these precious beings can harmonize well, we can do unimaginable things.  
  
I believe.  

![](https://theclassicalnovice.files.wordpress.com/2015/05/dublin_philharmonic_orchestra_performing_tchaikovskys_symphony_no_4_in_charlotte_north_carolina-e1431911721232.jpg?w=1180&h=435&crop=1)

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
!pip install ngboost

In [ ]:
import pandas as pd
import numpy as np

# encoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.preprocessing import StandardScaler, MinMaxScaler, scale
from sklearn.preprocessing import PolynomialFeatures

# cross validation.. etc

from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import KFold, StratifiedKFold
from collections import Counter

# ML
from sklearn.linear_model import LinearRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn import tree
from sklearn import ensemble
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor
from ngboost import NGBRegressor
import xgboost as xgb

from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import precision_recall_curve, roc_curve, auc
from sklearn.metrics import mean_absolute_error, mean_squared_error

print("Setup Complete")

In [ ]:
data_dir = '../input/petfinder-pawpularity-score'

In [ ]:
train = pd.read_csv(data_dir + '/train.csv')
test = pd.read_csv(data_dir + '/test.csv')
sample_submission = pd.read_csv(data_dir + '/sample_submission.csv')

In [ ]:
train.shape, test.shape, sample_submission.shape

In [ ]:
train.info()

In [ ]:
train.head()

In [ ]:
# df = train.drop(['Pawpularity'], axis=1)
# df_label = train['Pawpularity'].copy()

In [ ]:
# label - feature correlation 

df_matrix = train.corr()
df_matrix['Pawpularity'].sort_values(ascending=False)

In [ ]:
# correlation : Group, Accessory, Face / Blur, Subject Focus, Eyes 
attributes = ['Subject Focus', 'Near', 'Action', 'Collage', 'Info', 'Blur', 'Pawpularity'] # i got insight by looking at the scatterplot 
attributes_corr = ['Subject Focus', 'Eyes', 'Face', 'Accessory', 'Group', 'Blur', 'Pawpularity']

### Use all features

In [ ]:
# train_label = train['Pawpularity'].copy()

features = [
    f for f in train.columns if f not in ('Id')
]
train = train[features]
train.head()

### Label check
Please refer to my other kernel : https://www.kaggle.com/kimalpha/petfinder-simple-t-sne-fireworks

In [ ]:
# Ensemble 
from functools import partial
from scipy import optimize

class OptimizeRMSE:
    def __init__(self):
        self.coef_ = 0
        
    def _rmse(self, coef, X, y):
        '''
        :param coef : weight list 
        '''
        x_coef = X * coef
        predictions = np.sum(x_coef, axis=1)
        rmse = np.sqrt(mean_squared_error(y, predictions))
        
        return rmse
    
    def fit(self, X, y):
        loss_partial = partial(self._rmse, X=X, y=y)
        
        # any distribution where the sum of weights is 1 
        initial_coef = np.random.dirichlet(np.ones(X.shape[1]), size=1)
        self.coef_ = optimize.fmin(loss_partial, initial_coef, disp=True)
    
    def predict(self, X):
        x_coef = X * self.coef_
        predictions = np.sum(x_coef, axis=1)
        
        return predictions

In [ ]:
X = train.iloc[:, :-1].values
y = train.iloc[:, -1].values

xfold1, xfold2, yfold1, yfold2 = model_selection.train_test_split(
    X,
    y,
    test_size=0.5,
    stratify=y,
    shuffle=True,
    random_state=42
)

poly2 = PolynomialFeatures(degree=1)
lr_xfold1 = poly2.fit_transform(xfold1)

lr = LinearRegression()

# Best: {'criterion': 1, 'max_depth': 12.0, 'max_features': 1, 'min_samples_leaf': 1.0, 'min_samples_split': 4.0}
rf = RandomForestRegressor(random_state=42, n_jobs=-1,
                           criterion='mae',
                           max_depth=12,
                           max_features='sqrt',
                           min_samples_leaf=1,
                           min_samples_split=4
                          )                       

xgbr = xgb.XGBRegressor(random_state=42, 
                        n_jobs=-1,
                        use_label_encoder=True,
                        eval_metric='rmse',
                       )

ngb = NGBRegressor()

lr.fit(lr_xfold1, yfold1)
rf.fit(xfold1, yfold1)
xgbr.fit(xfold1, yfold1)
ngb.fit(xfold1, yfold1)

pred_lr = lr.predict(poly2.fit_transform(xfold2))
pred_rf = rf.predict(xfold2)
pred_xgbr = xgbr.predict(xfold2)
pred_ngb = ngb.predict(xfold2)
avg_pred = (pred_lr + pred_rf + pred_xgbr + pred_ngb) / 4

fold2_preds = np.column_stack((
    pred_lr,
    pred_rf,
    pred_xgbr,
    pred_ngb,
    avg_pred
))

rmse_fold2 = []

for i in range(fold2_preds.shape[1]):
    rmse = np.sqrt(mean_squared_error(yfold2, fold2_preds[:, i]))
    rmse_fold2.append(rmse)
    
print(f"Linear Regression RMSE : {rmse_fold2[0]}")
print(f"RandomForest Regressor RMSE : {rmse_fold2[1]}")
print(f"XGBoost Regressor RMSE: {rmse_fold2[2]}")
print(f"NGBoost Regressor RMSE: {rmse_fold2[3]}")

print(f"=================================================")
# fold change

poly2 = PolynomialFeatures(degree=1)
lr_xfold2 = poly2.fit_transform(xfold2)

lr = LinearRegression()

# Best: {'criterion': 1, 'max_depth': 12.0, 'max_features': 1, 'min_samples_leaf': 1.0, 'min_samples_split': 4.0}
rf = RandomForestRegressor(random_state=42, n_jobs=-1,
                           criterion='mae',
                           max_depth=12,
                           max_features='sqrt',
                           min_samples_leaf=1,
                           min_samples_split=4
                          )                       

xgbr = xgb.XGBRegressor(random_state=42, 
                        n_jobs=-1,
                        use_label_encoder=True,
                        eval_metric='rmse',
                       )

ngb = NGBRegressor()

lr.fit(lr_xfold2, yfold2)
rf.fit(xfold2, yfold2)
xgbr.fit(xfold2, yfold2)
ngb.fit(xfold2, yfold2)

pred_lr = lr.predict(poly2.fit_transform(xfold1))
pred_rf = rf.predict(xfold1)
pred_xgbr = xgbr.predict(xfold1)
pred_ngb = ngb.predict(xfold2)
avg_pred = (pred_lr + pred_rf + pred_xgbr + pred_ngb) / 4

fold1_preds = np.column_stack((
    pred_lr,
    pred_rf,
    pred_xgbr,
    pred_ngb,
    avg_pred
))

rmse_fold1 = []

for i in range(fold1_preds.shape[1]):
    rmse = np.sqrt(mean_squared_error(yfold1, fold1_preds[:, i]))
    rmse_fold1.append(rmse)
    
print(f"Linear Regression RMSE : {rmse_fold1[0]}")
print(f"RandomForest Regressor RMSE : {rmse_fold1[1]}")
print(f"XGBoost Regressor RMSE: {rmse_fold1[2]}")
print(f"NGBoost Regressor RMSE: {rmse_fold1[3]}")

opt = OptimizeRMSE()
opt.fit(fold1_preds[:, :-1], yfold1)
opt_preds_fold2 = opt.predict(fold2_preds[:, :-1])
rmse = np.sqrt(mean_squared_error(yfold2, opt_preds_fold2))

print(f"Optimized RMSE, Fold 2 = {rmse}")
print(f"Coefficients = {opt.coef_}")

opt = OptimizeRMSE()
opt.fit(fold2_preds[:, :-1], yfold2)
opt_preds_fold1 = opt.predict(fold1_preds[:, :-1])
rmse = np.sqrt(mean_squared_error(yfold1, opt_preds_fold1))

print(f"Optimized RMSE, Fold 1 = {rmse}")
print(f"Coefficients = {opt.coef_}")

### Feature selection

* Remove low distributed features. (easy way)  
* Remove features that have a high correlation. (easy way)      
* **Univariate Feature Selection: How to prenuclearize features based on each feature's statistical score in the target variable** (mutual information, ANOVA, f-test, chi square)  

In [ ]:
X = train.iloc[:, :-1]
y = train.iloc[:, -1]

In [ ]:
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import SelectPercentile

In [ ]:
class UnivariateFeatureSelection:
    def __init__(self, n_features, problem_type, scoring):
        if problem_type == "classification":
            valid_scoring = {
                "f_classif": f_classif,
                "chi2": chi2,
                "mutual_info_classif": mutual_info_classif
            }
        else:
            valid_scoring = {
                "f_regression": f_regression,
                "mutual_info_regression": mutual_info_regression
            }
        
        if scoring not in valid_scoring:
            raise Exception("Invalid scoring function, but you are so beautiful :)")
        
        if isinstance(n_features, int):
            self.selection = SelectKBest(
                valid_scoring[scoring],
                k=n_features
            )
            
        elif isinstance(n_features, float):
            self.selection = SelectPercentile(
                valid_scoring[scoring],
                percentile = int(n_features * 100)
            )
        
        else:
            raise Exception("Invalid type of feature")
    
    def fit(self, X, y):
        return self.selection.fit(X, y)
    
    def transform(self, X):
        return self.selection.transform(X)
    
    def fit_transform(self, X, y):
        return self.selection.fit_transform(X, y)

In [ ]:
train.head(1) # 12 features, 1 label 

In [ ]:
# what if you want to choose 50 % top features from the classification? 

ufs = UnivariateFeatureSelection(
    n_features=0.5,
    problem_type="classification",
    scoring='f_classif'
)
ufs.fit(X, y)
X_transformed = ufs.transform(X)
X_transformed.shape # select 12 --> 6 features (50 %)

In [ ]:
# what if you want to choose 10 top features from the regression? 

ufs = UnivariateFeatureSelection(
    n_features=10,
    problem_type="regression",
    scoring='f_regression'
)
ufs.fit(X, y)
X_transformed = ufs.transform(X)
X_transformed.shape # select 12 --> 10 features 

In [ ]:
total_x_trans = []

for n_features in range(1, X.shape[1]):
    ufs = UnivariateFeatureSelection(
        n_features=n_features,
        problem_type="regression",
        scoring='f_regression'
    )
    ufs.fit(X, y)
    X_transformed = ufs.transform(X)
    total_x_trans.append(X_transformed)

total_x_trans[0]

I want to find the number of features that minimize the loss function (RMSE) using a single model (NGBoost).  
  
so, the univariate feature selection and the greedy feature selection were combined.  
  
**@@@Caution@@@**  
Please note that the below code **takes a long long time** to execute because the two classes are combined.  
  
Depending on the range of choices, Calculate the loss function from 1 to 12 features. 

In [ ]:
# For the best feature selection, A function that calculates RMSE according to the number of features with a single ml model.(NGBoost)

# Modifying the code.

# class ML_FeatureSelection:
#     def rmse_score(self, X, y):
#         model = NGBRegressor() # no hyper-parameter tune 
#         model.fit(X, y)
#         preds = model.predict(X)
#         rmse = np.sqrt(mean_squared_error(y, preds))
        
#         return rmse
    
#     def best_feature_selection(self, X, y):
#         '''
#         param X : np array
#         param y : np array
#         return : best score, best features
#         '''
#         best_features = []
#         best_scores = []
        
#         num_features = X.shape[1]
        
#         while True:
#             first_feature = None
#             best_score = 0
            
#             for feature in range(num_features):
#                 if feature in best_features:
#                     continue
                
#                 selected_features = best_features + [feature]
#                 xtrain = X[:, selected_features]
#                 score = self.rmse_score(xtrain, y)
                
#                 if score > best_score:
#                     first_feature = feature
#                     best_score = score
            
#             if first_feature != None:
#                 best_features.append(first_feature)
#                 best_scores.append(best_score)
            
#             if len(best_scores) > 2:
#                 if best_scores[-1] < best_scores[-2]:
#                     break
                    
#         return best_scores[:-1], best_features[:-1]
    
#     def __call__(self, X, y):
#         scores, features = self.best_feature_selection(X, y)
        
#         return X[:, features], scores

# if __name__ == '__main__':
#     for i in range(X.shape[1]):
#         X = total_x_trans[i]
#         X_transformed, scores = ML_FeatureSelection()(X, y)

## Continue ~